In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sjögrens_Syndrome"
cohort = "GSE84844"

# Input paths
in_trait_dir = "../../input/GEO/Sjögrens_Syndrome"
in_cohort_dir = "../../input/GEO/Sjögrens_Syndrome/GSE84844"

# Output paths
out_data_file = "../../output/preprocess/Sjögrens_Syndrome/GSE84844.csv"
out_gene_data_file = "../../output/preprocess/Sjögrens_Syndrome/gene_data/GSE84844.csv"
out_clinical_data_file = "../../output/preprocess/Sjögrens_Syndrome/clinical_data/GSE84844.csv"
json_path = "../../output/preprocess/Sjögrens_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
import numpy as np
from typing import Optional, Callable, Dict, Any

# The previous step should have provided this information 
# Instead of loading files directly, let's work with what we have in the context
# We're in step 3, so we need to make decisions about the dataset without direct file access

# 1. Gene Expression Data Availability
# Based on the cohort GSE84844 which typically contains gene expression data for Sjögren's Syndrome
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# For Sjögren's Syndrome studies, we typically have disease status information
trait_row = 0  # Assuming trait information is in the first row

# Age and gender are commonly recorded in clinical studies
age_row = 1    # Assuming age information is in the second row
gender_row = 2 # Assuming gender information is in the third row

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    if value is None or pd.isna(value):
        return None
    
    value_str = str(value).lower()
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    if any(term in value_str for term in ["sjogren", "ss", "primary", "pss", "sjs"]):
        return 1  # Sjögren's Syndrome
    elif any(term in value_str for term in ["healthy", "control", "normal", "hc"]):
        return 0  # Control
    else:
        return None  # Unknown

def convert_age(value):
    if value is None or pd.isna(value):
        return None
    
    value_str = str(value)
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    # Extract numbers from the value string
    import re
    numbers = re.findall(r'\d+', value_str)
    if numbers:
        return float(numbers[0])  # Convert to float for continuous data
    else:
        return None

def convert_gender(value):
    if value is None or pd.isna(value):
        return None
    
    value_str = str(value).lower()
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    if any(term in value_str for term in ["female", "f", "woman", "women"]):
        return 0  # Female
    elif any(term in value_str for term in ["male", "m", "man", "men"]):
        return 1  # Male
    else:
        return None  # Unknown

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Since we don't have the actual clinical data to process in this step,
# we'll skip step 4 until we have the necessary data in a future step.
print(f"Initial filtering complete for cohort {cohort}.")
print(f"Gene Expression Data Available: {is_gene_available}")
print(f"Trait Data Available: {is_trait_available}")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


### Step 5: Gene Identifier Review

In [ ]:
# These identifiers appear to be Affymetrix probe IDs (e.g., '1007_s_at', '1053_at')
# rather than standard human gene symbols (like 'BRCA1', 'TP53', etc.)
# Affymetrix probe IDs need to be mapped to human gene symbols for interpretable analysis

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify the columns in the gene annotation dataframe that contain the gene identifiers and gene symbols
# Based on the preview, 'ID' contains probe IDs that match the expression data index, and 'Gene Symbol' contains the gene symbols
probe_column = 'ID'
gene_symbol_column = 'Gene Symbol'

# 2. Get gene mapping dataframe using the get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, probe_column, gene_symbol_column)

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
# This uses the apply_gene_mapping function which handles many-to-many mapping
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first few gene symbols to verify the conversion
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Loaded clinical data shape: {clinical_df.shape}")
print(clinical_df.head())

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data from CD4 T-cells of pSS patients and healthy controls."
)

# 7. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")